In [1]:
import math
import pickle
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten   
from tensorflow.keras.layers import Layer, ZeroPadding2D, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

### Helpers

In [2]:
# an implication of Pytorch CrossMapLRN2d with Keras
class LRN2D(Layer):
    """
    This code is adapted from pylearn2.
    License at: https://github.com/lisa-lab/pylearn2/blob/master/LICENSE.txt
    """

    def __init__(self, alpha=1e-4, k=2, beta=0.75, n=5):
        if n % 2 == 0:
            raise NotImplementedError('LRN2D only works with odd n. n provided: ' + str(n))
        super(LRN2D, self).__init__()
        self.alpha = alpha
        self.k = k
        self.beta = beta
        self.n = n

    def get_output(self, train):
        X = self.get_input(train)
        b, ch, r, c = X.shape
        half_n = self.n // 2
        input_sqr = T.sqr(X)
        extra_channels = T.alloc(0., b, ch + 2*half_n, r, c)
        input_sqr = T.set_subtensor(extra_channels[:, half_n:half_n+ch, :, :], input_sqr)
        scale = self.k
        for i in range(self.n):
            scale += self.alpha * input_sqr[:, i:i+ch, :, :]
        scale = scale ** self.beta
        return X / scale

    def get_config(self):
        return {'name': self.__class__.__name__,
                'alpha': self.alpha,
                'k': self.k,
                'beta': self.beta,
                'n': self.n}

    

# another implication of Pytorch CrossMapLRN2d with Keras
class LocalResponseNormalization(Layer):
  
    def __init__(self, n=5, alpha=1e-4, beta=0.75, k=2, **kwargs):
        self.n = n
        self.alpha = alpha
        self.beta = beta
        self.k = k
        super(LocalResponseNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.shape = input_shape
        super(LocalResponseNormalization, self).build(input_shape)

    def call(self, x):
        _, r, c, f = self.shape 
        squared = K.square(x)
        pooled = K.pool2d(squared, (self.n, self.n), strides=(1,1), padding='same', pool_mode='avg')
        summed = K.sum(pooled, axis=3, keepdims=True)
        averaged = self.alpha * K.repeat_elements(summed, f, axis=3)
        denom = K.pow(self.k + averaged, self.beta)
        return x / denom 
    
    def compute_output_shape(self, input_shape):
        return input_shape



# Function for changing learning rate over epochs
def lr_decay(epoch):
    decay = .1
    lr = base_lr * math.pow(decay, (epoch // 30))   
    return lr


## **1. Data**

In [3]:
# load the scaled dataset
path = 'D:/ETE/Data/Data_Mini/'
pickles = ['LeftEye_h', 'RightEye_h', 'Face_h', 'GridPaper', 'labels']
names = ['eye_left', 'eye_right', 'face', 'grid', 'labels']

for idx, pkl in tqdm(enumerate(pickles)):
    with open(path + pkl + '.pkl','rb') as f:
        exec(names[idx] + ' = pickle.load(f)')


In [4]:
for name in names:
    exec('print(name, \'is a\', type(' + name + '), \'with shape:\', ' + name + '.shape)')
    exec('print(name, \'ranage\',' + name + '.min(), \'-\',' + name + '.max())')
    print('\n')


eye_left is a <class 'numpy.ndarray'> with shape: (9994, 112, 112, 3)
eye_left ranage 0 - 255


eye_right is a <class 'numpy.ndarray'> with shape: (9994, 112, 112, 3)
eye_right ranage 0 - 255


face is a <class 'numpy.ndarray'> with shape: (9994, 112, 112, 3)
face ranage 0 - 255


grid is a <class 'numpy.ndarray'> with shape: (9994, 25, 25)
grid ranage 0 - 1


labels is a <class 'numpy.ndarray'> with shape: (9994, 2)
labels ranage -26.5163572474 - 7.01304509963




In [9]:
eye_left = eye_left / 255
eye_left = eye_left.astype(np.float32)
eye_right = eye_right / 255
eye_right = eye_right.astype(np.float32)
face = face / 255
face = face.astype(np.float32)

## **2. The Model & Training**

In [10]:
# --------------------------------------------------------------------------
# Eye Net
# --------------------------------------------------------------------------
eye_input = Input(shape=(112,112,3))
eye_cnn1_out = Conv2D(filters=96,
                       kernel_size=11,
                       strides=4,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4),
                       input_shape=(224,224,3))(eye_input)
eye_max1_out = MaxPool2D(pool_size=3, strides=2)(eye_cnn1_out)
eye_lrn1_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eye_max1_out)
eye_zro1_out = ZeroPadding2D(padding=2)(eye_lrn1_out)
eye_cnn2_out = Conv2D(filters=256,
                      kernel_size=5,
                      strides=1,
                      padding='valid',
                      #groups=2,
                      activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(eye_zro1_out)
eye_max2_out = MaxPool2D(pool_size=3, strides=2)(eye_cnn2_out)
eye_lrn2_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(eye_max2_out)
eye_cnn3_out = Conv2D(filters=384,
                      kernel_size=3,
                      strides=1,
                      padding='same',
                      activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(eye_lrn2_out)
eye_cnn4_out = Conv2D(filters=64,
                      kernel_size=1,
                      strides=1,
                      padding='valid',
                      activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(eye_cnn3_out)
eye_out = Flatten()(eye_cnn4_out)

model_eye = Model(eye_input, eye_out)



# --------------------------------------------------------------------------
# Face Net
# --------------------------------------------------------------------------
face_input = Input(shape=(112,112,3))
face_cnn1_out = Conv2D(filters=96,
                       kernel_size=11,
                       strides=4,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4),
                       input_shape=(224,224,3))(face_input)
face_max1_out = MaxPool2D(pool_size=3, strides=2)(face_cnn1_out)
face_lrn1_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(face_max1_out)
face_zro1_out = ZeroPadding2D(padding=2)(face_lrn1_out)
face_cnn2_out = Conv2D(filters=256,
                       kernel_size=5,
                       strides=1,
                       padding='valid',
                       #groups=2,
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(face_zro1_out)
face_max2_out = MaxPool2D(pool_size=3, strides=2)(face_cnn2_out)
face_lrn2_out = LRN2D(n=5, alpha=1e-4, beta=0.75, k=1.0)(face_max2_out)
face_cnn3_out = Conv2D(filters=384,
                       kernel_size=3,
                       strides=1,
                       padding='same',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(face_lrn2_out)
face_cnn4_out = Conv2D(filters=64,
                       kernel_size=1,
                       strides=1,
                       padding='valid',
                       activation='relu',
                       kernel_regularizer=regularizers.l2(1e-4))(face_cnn3_out)
face_flt_out = Flatten()(face_cnn4_out)
face_dns_out = Dense(128, activation = 'relu',
                     kernel_regularizer=regularizers.l2(1e-4))(face_flt_out)
face_out = Dense(64, activation = 'relu')(face_dns_out)



# --------------------------------------------------------------------------
# Grids' Net
# --------------------------------------------------------------------------
grid_input = Input(shape=(25, 25)) # Paper Grid
# grid_input = Input(shape=(320, 240)) # MTCNN Grid
grid_flt_out = Flatten()(grid_input)
grid_dns_out = Dense(256, activation = 'relu',
                     kernel_regularizer=regularizers.l2(1e-4))(grid_flt_out)
grid_out = Dense(128, activation = 'relu',
                 kernel_regularizer=regularizers.l2(1e-4))(grid_dns_out)



# --------------------------------------------------------------------------
# Join Eyes' Nets
# --------------------------------------------------------------------------
eye_left_input = Input(shape=(112,112,3))
eye_right_input = Input(shape=(112,112,3))
eye_left_out = model_eye(eye_left_input)
eye_right_out = model_eye(eye_right_input)

concat_eyes = concatenate([eye_left_out, eye_right_out])

eyes_out = Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(1e-4))(concat_eyes)



# --------------------------------------------------------------------------
# Join All Nets
# --------------------------------------------------------------------------
concat_all = concatenate([eyes_out, face_out, grid_out])

final_dns_out = Dense(128, activation='relu',
                      kernel_regularizer=regularizers.l2(1e-4))(concat_all)
final_out = Dense(2, activation='relu',
                  kernel_regularizer=regularizers.l2(1e-4))(final_dns_out)

final_model = Model([eye_left_input, eye_right_input, face_input, grid_input], final_out)



# --------------------------------------------------------------------------
# training
# --------------------------------------------------------------------------
base_lr = 0.0001
momentum = 0.9

sgd = SGD(lr=base_lr, momentum=momentum)

lr_callback = LearningRateScheduler(lr_decay)
checkpointer = ModelCheckpoint(filepath='Weights.hdf5', monitor='val_loss',
                               verbose=1, save_best_only=True)

final_model.compile(optimizer=sgd, loss='mae', metrics=['mae'])

final_model.fit(x=[eye_left, eye_right, face, grid], 
                y=labels,
                # x=[eye_left[:8000,:,:,:], eye_right[:8000,:,:,:],
                #    face[:8000,:,:,:], grid[:8000,:,:]], 
                # y=labels[:8000],
                batch_size=32,
                epochs=10,
                validation_split=0.2,
                # validation_data=([eye_left[8000:,:,:,:], eye_right[8000:,:,:,:],
                #                   face[8000:,:,:,:], grid[8000:,:,:]], labels[8000:]),
                shuffle=True,
                callbacks=[lr_callback, checkpointer])


Train on 7995 samples, validate on 1999 samples
Epoch 1/10
7968/7995 [============================>.] - ETA: 0s - loss: 4.2978 - mae: 4.0647  ETA - ETA: 6s  - 
Epoch 00001: val_loss improved from inf to 4.17005, saving model to Weights.hdf5
7995/7995 [==============================] - 13s 2ms/sample - loss: 4.2966 - mae: 4.0634 - val_loss: 4.1701 - val_mae: 3.9369
Epoch 2/10
7968/7995 [============================>.] - ETA: 0s - loss: 4.2629 - mae: 4.0298- ETA: 8s - loss: 4.0 - ETA: 4s - loss: 4.2542 - mae: - ETA: 3s - loss: 4.2694  - ETA: 3s - loss: 4.2542 - mae:  - ETA: 2s - loss: 4.2580  - ETA: 1s - loss: 4.2546  - ETA: 0s - loss: 4.2685 - mae: 4.035 - ETA: 0s - loss: 4.2679 - mae: 4.034 - ETA: 0s - loss: 4.2684 - m - ETA: 0s - loss: 4.2618 - mae: 4.02
Epoch 00002: val_loss improved from 4.17005 to 4.16276, saving model to Weights.hdf5
7995/7995 [==============================] - 10s 1ms/sample - loss: 4.2641 - mae: 4.0310 - val_loss: 4.1628 - val_mae: 3.9297
Epoch 3/10
7968/7995 [=